In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import time

# 预设参数
CLASS_NUM = 10
BATCH_SIZE = 128
EPOCH = 5

# 检验GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cuda:0


In [2]:
cifar10 = torchvision.datasets.CIFAR10(
    root='D:\zju\kaixuehou\data',
    train=True,
    download=True,
    transform=transform
)
cifar10_test = torchvision.datasets.CIFAR10(
    root='D:\zju\kaixuehou\data',
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)
print(cifar10)
print(cifar10_test)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Split: train
    Root Location: D:\zju\kaixuehou\data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None
Dataset CIFAR10
    Number of datapoints: 10000
    Split: test
    Root Location: D:\zju\kaixuehou\data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None


In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.conv5 = nn.Conv2d(128,128, 3,padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv7 = nn.Conv2d(128, 128, 1,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

        self.conv8 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv9 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, 1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()

        self.conv11 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, 1, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        self.fc14 = nn.Linear(512*4*4,1024)
        self.drop1 = nn.Dropout2d()
        self.fc15 = nn.Linear(1024,1024)
        self.drop2 = nn.Dropout2d()
        self.fc16 = nn.Linear(1024,10)

    def forward(self,x):
        x = x.to(device)  # 自加
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        # print(" x shape ",x.size())
        x = x.view(-1,512*4*4)
        x = F.relu(self.fc14(x))
        x = self.drop1(x)
        x = F.relu(self.fc15(x))
        x = self.drop2(x)
        x = self.fc16(x)

        return x



In [4]:
net = Net()
net = net.load_state_dict(torch.load('lenet_cifar_10.model'))
net.to(device)
    
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=5e-4) 
loss_fn = nn.CrossEntropyLoss()

In [5]:
loss_list = []
Accuracy = []

for epoch in range(1, EPOCH + 1):
    timestart = time.time()
    for step, (x, y) in enumerate(train_loader):
        b_x = x.to(device)
        b_y = y.to(device)
        output = net(b_x)
        loss = loss_fn(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 50 == 0:
            loss_list.append(loss)
            print('step: %d ' % step)
    pre_correct = 0.0
    for (x, y) in (test_loader):
        b_x, b_y = x.to(device), y.to(device) 
        output = net(b_x)
        pre = torch.max(output, 1)[1]
        pre_correct = pre_correct + float(torch.sum(pre == b_y))
    print('EPOCH:{epoch},ACC:{acc}%'.format(epoch=epoch, acc=(pre_correct / float(10000)) * 100))
    Accuracy.append(pre_correct / float(10000) * 100)
    print('epoch %d cost %3f sec' % (epoch, time.time() - timestart))
torch.save(net.state_dict() , 'lenet_cifar_10.model')

step: 0 
step: 50 
step: 100 
step: 150 
step: 200 
step: 250 
step: 300 
step: 350 
EPOCH:1,ACC:24.16%
epoch 1 cost 105.827702 sec
step: 0 
step: 50 
step: 100 
step: 150 
step: 200 
step: 250 
step: 300 
step: 350 
EPOCH:2,ACC:36.08%
epoch 2 cost 104.779115 sec
step: 0 
step: 50 
step: 100 
step: 150 
step: 200 
step: 250 
step: 300 
step: 350 
EPOCH:3,ACC:45.32%
epoch 3 cost 104.828912 sec
step: 0 
step: 50 
step: 100 
step: 150 
step: 200 
step: 250 
step: 300 
step: 350 
EPOCH:4,ACC:52.900000000000006%
epoch 4 cost 103.967363 sec
step: 0 
step: 50 
step: 100 
step: 150 
step: 200 
step: 250 
step: 300 
step: 350 
EPOCH:5,ACC:56.63%
epoch 5 cost 103.460542 sec
step: 0 
step: 50 
step: 100 
step: 150 
step: 200 
step: 250 
step: 300 
step: 350 
EPOCH:6,ACC:61.029999999999994%
epoch 6 cost 104.182611 sec
step: 0 
step: 50 
step: 100 
step: 150 
step: 200 
step: 250 
step: 300 
step: 350 
EPOCH:7,ACC:63.65%
epoch 7 cost 104.267983 sec
step: 0 
step: 50 
step: 100 
step: 150 
step: 200